# Preparing the Data
## Importing packages

In [31]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
import os

## Reading in data

In [32]:
project_root = os.getcwd()  # Assumes script runs from project root

# Define the path to the processed data folder
data_path = os.path.join(project_root, "../data", "processed")

# Load the CSV files
customer_df = pd.read_csv(os.path.join(data_path, "customer.csv"))
digital_usage_df = pd.read_csv(os.path.join(data_path, "digital_usage.csv"))
transactions_df = pd.read_csv(os.path.join(data_path, "transactions.csv"))
loans_df = pd.read_csv(os.path.join(data_path, "loans.csv"))
products_df = pd.read_csv(os.path.join(data_path, "products_owned.csv"))

## Ensure dates are in the right format

In [33]:
transactions_df["transaction_date"] = pd.to_datetime(transactions_df["transaction_date"])
loans_df["due_date"] = pd.to_datetime(loans_df["due_date"])
loans_df["paid_off_date"] = pd.to_datetime(loans_df["paid_off_date"])

## Creating interaction terms

### Transaction Table
Since days_from_last_transaction is calculated using the current date so this particular value will be changing everyday.

In [34]:
# Calculate days_from_last_transaction
latest_transaction = transactions_df.groupby("customer_id")["transaction_date"].max().reset_index()
latest_transaction["days_from_last_transaction"] = (pd.to_datetime("today") - latest_transaction["transaction_date"]).dt.days
latest_transaction = latest_transaction[["customer_id", "days_from_last_transaction"]]
# Calculate avg_transaction_amt per customer
transaction_summary = transactions_df.groupby("customer_id").agg(total_transaction_amt=("transaction_amt", "sum"),num_transactions=("transaction_id", "count")).reset_index()
transaction_summary["avg_transaction_amt"] = transaction_summary["total_transaction_amt"] / transaction_summary["num_transactions"]
transaction_summary = transaction_summary[["customer_id", "avg_transaction_amt"]]

### Digital Usage table

In [35]:
# Calculate digital engagement score
scaler = MinMaxScaler()
digital_usage_df["normalized_logins"] = scaler.fit_transform(digital_usage_df[["mobile_logins_wk", "web_logins_wk"]].sum(axis=1).values.reshape(-1, 1))
digital_usage_df["normalized_session_time"] = scaler.fit_transform(digital_usage_df[["avg_mobile_time", "avg_web_time"]].sum(axis=1).values.reshape(-1, 1))
digital_usage_df["digital_engagement_score"] = (digital_usage_df["normalized_logins"] * 0.7 +digital_usage_df["normalized_session_time"] * 0.3)
# Correct aggregation step
digital_engagement = digital_usage_df.groupby("customer_id", as_index=False)["digital_engagement_score"].mean()

### Loans table

In [36]:
# Median loan repayment time per customer
loans_df["loan_repayment_time"] = (loans_df["paid_off_date"] - loans_df["due_date"]).dt.days
loan_summary = loans_df.groupby("customer_id")["loan_repayment_time"].median().reset_index()

### Products table

In [37]:
# Total products owned
products_df["total_products_owned"] = products_df.iloc[:, 1:].sum(axis=1)
products_summary = products_df[["customer_id", "total_products_owned"]]

### Customer table

In [38]:
# Extracting relevant columns from customer table
customer_features = ['customer_id', 'income', 'balance', 'customer_lifetime_value', 'debt', 'tenure', 'default']
customer_subset_df = customer_df[customer_features]

## Merging datasets

In [39]:
df = customer_subset_df.merge(latest_transaction, on="customer_id", how="left")
df = df.merge(transaction_summary[["customer_id", "avg_transaction_amt"]], on="customer_id", how="left")
df = df.merge(digital_engagement[["customer_id", "digital_engagement_score"]], on="customer_id", how="left")
df = df.merge(loan_summary, on="customer_id", how="left")
df = df.merge(products_df[["customer_id", "total_products_owned"]], on="customer_id", how="left")
print(df.head())

   customer_id   income   balance  customer_lifetime_value      debt  tenure  \
0          166  1456.22 -10837.70                   214.12  38473.44      31   
1          139   597.64  -8163.90                   255.79  96402.54      20   
2          484   822.64  -7492.65                   129.42   9731.79      36   
3          508   340.44  -5520.04                   125.30  18799.09      29   
4          119  1022.64  -5071.99                   347.16  19080.59      36   

   default  days_from_last_transaction  avg_transaction_amt  \
0        1                         NaN                  NaN   
1        1                         NaN                  NaN   
2        1                         NaN                  NaN   
3        1                       250.0              109.536   
4        1                         NaN                  NaN   

   digital_engagement_score  loan_repayment_time  total_products_owned  
0                  0.417834               -627.5                   

In [40]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_id                 4000 non-null   int64  
 1   income                      4000 non-null   float64
 2   balance                     4000 non-null   float64
 3   customer_lifetime_value     4000 non-null   float64
 4   debt                        4000 non-null   float64
 5   tenure                      4000 non-null   int64  
 6   default                     4000 non-null   int64  
 7   days_from_last_transaction  1844 non-null   float64
 8   avg_transaction_amt         1844 non-null   float64
 9   digital_engagement_score    3981 non-null   float64
 10  loan_repayment_time         2629 non-null   float64
 11  total_products_owned        4000 non-null   int64  
dtypes: float64(8), int64(4)
memory usage: 375.1 KB
None


In [41]:
print(df.isnull().sum())

customer_id                      0
income                           0
balance                          0
customer_lifetime_value          0
debt                             0
tenure                           0
default                          0
days_from_last_transaction    2156
avg_transaction_amt           2156
digital_engagement_score        19
loan_repayment_time           1371
total_products_owned             0
dtype: int64


## Handling missing values
`digital_engagement_score` only has 19 missing values so we can use KNN Inputer
`avg_transaction_amt` is NaN means no transaction data found so we set NaN to be 0
`days_from_last_transaction` is NaN means no transaction data. We set last transaction number of days to be large but not too extreme. 
`has_loan` is NaN means no data on loans taken for that customer, We set NaN values to be 0. For those who has loaned before but but those with loans but no date of payment, we set the NaN values to be large but not too extreme


In [42]:

### engagement score only has 19 missing values
df["digital_engagement_score"] = KNNImputer(n_neighbors=5).fit_transform(df[["digital_engagement_score"]])
### no transaction record (we set transactions to be 0)
df["avg_transaction_amt"].fillna(0, inplace=True)
### some did not do transactions. we set the days from last transaction to be a high but not too extreme value. We add 2 months of buffer
df["days_from_last_transaction"].fillna(df["days_from_last_transaction"].max() + 30, inplace=True)
### clients who did not loan before, we set has_loan variable to 0. for those that has loaned but hasnt repaid will have repayment time of max repayment time + buffer of a month
df["has_loan"] = df["customer_id"].isin(loans_df["customer_id"]).astype(int)
loan_repay_max = df["loan_repayment_time"].max()
df["loan_repayment_time"] = df.apply(lambda row: 0 if row["has_loan"] == 0 else (loan_repay_max + 30 if np.isnan(row["loan_repayment_time"]) else row["loan_repayment_time"]),axis=1)

/var/folders/wc/5cwlt66x6d12kf_tz4qkd_580000gn/T/ipykernel_4895/337393323.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["avg_transaction_amt"].fillna(0, inplace=True)
/var/folders/wc/5cwlt66x6d12kf_tz4qkd_580000gn/T/ipykernel_4895/337393323.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [43]:
print(df.isnull().sum())

customer_id                   0
income                        0
balance                       0
customer_lifetime_value       0
debt                          0
tenure                        0
default                       0
days_from_last_transaction    0
avg_transaction_amt           0
digital_engagement_score      0
loan_repayment_time           0
total_products_owned          0
has_loan                      0
dtype: int64


# Standardizing

In [44]:
scaler = StandardScaler()
features_to_scale = ["income", "balance", "customer_lifetime_value", "debt", "days_from_last_transaction", "avg_transaction_amt", "digital_engagement_score", "total_products_owned", "loan_repayment_time"]
df_scaled = scaler.fit_transform(df[features_to_scale])
df_scaled = pd.DataFrame(df_scaled, columns=features_to_scale)

In [45]:
features = df.columns
print(features)

Index(['customer_id', 'income', 'balance', 'customer_lifetime_value', 'debt',
       'tenure', 'default', 'days_from_last_transaction',
       'avg_transaction_amt', 'digital_engagement_score',
       'loan_repayment_time', 'total_products_owned', 'has_loan'],
      dtype='object')


# Modeling

## K-Means Clustering
We use k = 4 to segment the customers into 4 categories 
1. High value
2. Occasional
3. Budget conscious
4. At risk / inactive customers
At risk / inactive customers

In [46]:
optimal_k = 4
df_scaled["Cluster"] = KMeans(n_clusters= optimal_k, random_state=42).fit_predict(df_scaled)
df["Cluster"] = df_scaled["Cluster"]

### Number of clients in each cluster

In [47]:
print(df["Cluster"].value_counts())

Cluster
0    1534
3    1365
2     912
1     189
Name: count, dtype: int64


### Getting information about each cluster's mean of features

In [48]:
cluster_means = df_scaled.groupby("Cluster")[features_to_scale].mean()
print(cluster_means)

           income   balance  customer_lifetime_value      debt  \
Cluster                                                          
0       -0.200312 -0.159079                -0.062185 -0.040794   
1        3.031960  2.885331                 0.305981  0.027274   
2       -0.210849 -0.160435                -0.087447  0.095988   
3       -0.053822 -0.113541                 0.085944 -0.022064   

         days_from_last_transaction  avg_transaction_amt  \
Cluster                                                    
0                          0.722719            -0.563809   
1                          0.138334            -0.042496   
2                          0.547439            -0.456242   
3                         -1.197113             0.944327   

         digital_engagement_score  total_products_owned  loan_repayment_time  
Cluster                                                                       
0                        0.018448              0.089732            -0.701264  
1    

## Cluster Labels
We assign weighted scores to each cluster

- 40% Income
- 20% Balance
- 20% Digital Engagement
- 10% Transactions
- 10% Products Owned
Then we rank clusters based on their total score

Top cluster = High-Value
2nd highest = Occasional
3rd highest = Budget-Conscious
Last = Inactive
Assign segments dynamically based on rankings

In [49]:
# Compute Weighted Scores
cluster_means["score"] = (
    cluster_means["income"] * 0.4 + 
    cluster_means["balance"] * 0.2 + 
    cluster_means["digital_engagement_score"] * 0.2 + 
    cluster_means["avg_transaction_amt"] * 0.1 + 
    cluster_means["total_products_owned"] * 0.1
)

# Rank Clusters Based on Score (Descending)
sorted_clusters = cluster_means["score"].sort_values(ascending=False).index.tolist()

# Assign Segments Based on Rank
dynamic_segment_mapping = {
    sorted_clusters[0]: "High-value",
    sorted_clusters[1]: "Occasional",
    sorted_clusters[2]: "Budget-conscious",
    sorted_clusters[3]: "At risk / inactive customers"
}

# Print cluster rankings before applying
print("\nCluster Ranking by Score (Best to Worst):")
for i, cluster in enumerate(sorted_clusters):
    print(f"Rank {i+1}: Cluster {cluster} → {dynamic_segment_mapping[cluster]}")


Cluster Ranking by Score (Best to Worst):
Rank 1: Cluster 1 → High-value
Rank 2: Cluster 3 → Occasional
Rank 3: Cluster 0 → Budget-conscious
Rank 4: Cluster 2 → At risk / inactive customers


# Creating DF with the segments

In [50]:
# Apply Mapping to DataFrame
df["Segment"] = df["Cluster"].map(dynamic_segment_mapping)
print(df["Segment"].value_counts())

Segment
Budget-conscious                1534
Occasional                      1365
At risk / inactive customers     912
High-value                       189
Name: count, dtype: int64


In [51]:
df_final = df[["customer_id", "Segment"]]
print(df_final.head())

   customer_id                       Segment
0          166              Budget-conscious
1          139  At risk / inactive customers
2          484              Budget-conscious
3          508  At risk / inactive customers
4          119  At risk / inactive customers


In [52]:
# Getting information of these segments
segment_means = df.groupby("Segment")[features_to_scale].mean()
# Display the results
print("Mean of original features per segment:")
print(segment_means)

Mean of original features per segment:
                                    income       balance  \
Segment                                                    
At risk / inactive customers   4247.079221    784.099386   
Budget-conscious               4280.716890    793.310398   
High-value                    14600.033122  21477.896772   
Occasional                     4748.399934   1102.706374   

                              customer_lifetime_value          debt  \
Segment                                                               
At risk / inactive customers               302.722292  27741.362818   
Budget-conscious                           306.303911  24371.915913   
High-value                                 358.503228  26048.675079   
Occasional                                 327.305941  24833.304755   

                              days_from_last_transaction  avg_transaction_amt  \
Segment                                                                         
At risk / i

## Creates csv table in under customer segmentation

In [55]:
df_final.to_csv("customer_segments.csv", index=False)
print("Saved 'customer_segments.csv' with Customer ID & segment name")

Saved 'customer_segments.csv' with Customer ID & segment name
